### Interact with transmission spectra of waveplate-polarizer filters in lower-dimensional space

Run All and scroll to bottom to play with sliders controlling the filter parameters (rotation angles of waveplate(s) and polarizer) and see the changes in the transmission spectra in a lower-dimensional space (from PCA).

In the cell after imports, set `basis_2d = True` to see a filter system with 1 waveplate and 1 polarizer (which is 2d in the lower-D space) or set `basis_2d = False` to see a filter system with 2 waveplates and 1 polarizer (which is 4d in the lower-D space).

12/29/2020

In [1]:
%cd ".."

import numpy as np
from sklearn.decomposition import PCA
import plotly.graph_objects as go
from ipywidgets import widgets, interact

from polarization import phase_shift, transmission_mueller

c:\Users\kate\hyperspectral


In [2]:
# Whether to do 1 waveplate layer (2d basis space) or 2 waveplate layers (4d basis space)
basis_2d = False

In [3]:
n_wav = 33
n_alpha = 20
n_theta = 20

wavelengths = np.linspace(400., 700., n_wav)
alphas = np.linspace(0., 180., n_alpha)
thetas = np.linspace(0., 180., n_theta)
gammas1 = [phase_shift(w, 66040, 0.01) for w in wavelengths]
gammas2 = [phase_shift(w, 78740, 0.01) for w in wavelengths]

if basis_2d:
    # Create dataset for 1 tape layer
    transmissions = np.zeros((n_alpha, n_theta, n_wav))

    for w in range(n_wav):
        for i in range(n_alpha):
            a = np.deg2rad(alphas[i])
            for j in range(n_theta):
                t = np.deg2rad(thetas[j])
                transmissions[i, j, w] = transmission_mueller(gammas1[w], a, t)

    transmissions = transmissions.reshape(-1, n_wav)

else:
    # Create dataset for 2 tape layers
    transmissions = np.zeros((n_alpha, n_alpha, n_theta, n_wav))

    for w in range(n_wav):
        for i in range(n_alpha):
            a1 = np.deg2rad(alphas[i])
            for j in range(n_alpha):
                a2 = np.deg2rad(alphas[j])
                for k in range(n_theta):
                    t = np.deg2rad(thetas[k])
                    transmissions[i, j, k, w] = transmission_mueller([gammas1[w], gammas2[w]], [a1, a2], t)

    transmissions = transmissions.reshape(-1, n_wav)

In [4]:
# Create basis functions from dataset
pca = PCA(n_components = 2 if basis_2d else 4)
pca.fit(transmissions)
pca_basis = pca.components_
print("pca explained variance ratio: ", end = "")
print(pca.explained_variance_ratio_)
print("sum of pca explained variance ratios: ", end = "")
print(np.sum(pca.explained_variance_ratio_))

# Project filter transmissions down to basis function coefficients
filter_coefs, _, _, _ = np.linalg.lstsq(pca_basis.transpose(), transmissions.transpose(), rcond=None)

pca explained variance ratio: [0.48061515 0.24101489 0.15368021 0.12431774]
sum of pca explained variance ratios: 0.9996279883197194


In [5]:
alpha1_text = "rotation angle of waveplate" if basis_2d else "rotation angle of waveplate 1"
alpha1_slider = widgets.IntSlider(value = 0, min = 0, max = 180, step = 1, layout = widgets.Layout(width = "100%"))
alpha1_label = widgets.Label(value = alpha1_text, layout = widgets.Layout(width = "250px"))
alpha1_box = widgets.HBox([alpha1_label, alpha1_slider])

alpha2_slider = widgets.IntSlider(value = 0, min = 0, max = 180, step = 1, layout = widgets.Layout(width = "100%"))
alpha2_label = widgets.Label(value = "rotation angle of waveplate 2", layout = widgets.Layout(width = "250px"))
alpha2_box = widgets.HBox([alpha2_label, alpha2_slider])

theta_slider = widgets.IntSlider(value = 0, min = 0, max = 180, step = 1, layout = widgets.Layout(width = "100%"))
theta_label = widgets.Label(value = "rotation angle of polarizer", layout = widgets.Layout(width = "250px"))
theta_box = widgets.HBox([theta_label, theta_slider])

ui = widgets.VBox([alpha1_box, theta_box]) if basis_2d else widgets.VBox([alpha1_box, alpha2_box, theta_box])

x_offset = np.min(filter_coefs[0, :] - 5)
y_offset = np.min(filter_coefs[1, :] - 5)

if not basis_2d:
    z_offset = np.min(filter_coefs[2, :] - 5)

if basis_2d:
    scatter_all = go.Scatter(
        x = filter_coefs[0, :], y = filter_coefs[1, :],
        mode = "markers",
        marker = dict(
            size = 5,
            color = "#cccccc",
            opacity = 0.3
        )
    )

    scatter_pt = go.Scatter(
        x = [filter_coefs[0, 0]], y = [filter_coefs[1, 0]],
        mode = "markers",
        marker = dict(
            size = 10,
            color = [filter_coefs[0, 0]],
            colorscale = "Plasma",
            cauto = False,
            cmin = -6.,
            cmax = 0.5,
            opacity = 1.0
        )
    )

    # spike_x = go.Scatter(
    #     x = [filter_coefs[0, 0], filter_coefs[0, 0]], y = [filter_coefs[1, 0], y_offset],
    #     mode = "lines",
    #     line = dict(
    #         width = 1,
    #         color = "black"
    #     )
    # )

    # spike_y = go.Scatter(
    #     x = [filter_coefs[0, 0], x_offset], y = [filter_coefs[1, 0], filter_coefs[1, 0]],
    #     mode = "lines",
    #     line = dict(
    #         width = 1,
    #         color = "black"
    #     )
    # )

    # spike_z = go.Scatter()
     
    scatter_trace = go.Scatter(
        x = np.repeat(filter_coefs[0, 0], repeats = 10),
        y = np.repeat(filter_coefs[1, 0], repeats = 10),
        mode = 'markers',
        marker = dict(
            size = 8,
            color = np.repeat(filter_coefs[0, 0], repeats = 10), 
            colorscale = 'Plasma',
            cauto = False,
            cmin = -6.,
            cmax = 0.5,
            opacity = 0.5
        )
    )

    layout = go.Layout(
        title = "dataset of filter transmissions projected onto PCA basis space",
        xaxis_title = "basis_0",
        xaxis_autorange = False,
        xaxis_range = [-6., 0.5],
        yaxis_title = "basis_1",
        yaxis_autorange = False,
        yaxis_range = [-1.7, 3.],
        showlegend = False
    )
else:
    scatter_all = go.Scatter3d(
        x = filter_coefs[0,:], y = filter_coefs[1,:], z = filter_coefs[2,:],
        mode = 'markers',
        marker = dict(
            size = 2,
            color = "#aaaaaa",
            # color = filter_coefs[3,:],
            # colorscale = 'Plasma',
            opacity = 0.1
        )
    )

    scatter_pt = go.Scatter3d(
        x = [filter_coefs[0,0]], y = [filter_coefs[1,0]], z = [filter_coefs[2,0]],
        mode = 'markers',
        marker = dict(
            size = 7,
            color = [filter_coefs[3,0]],
            colorbar = dict(
                title = 'basis_3'
            ),
            colorscale = 'Plasma',
            cauto = False,
            cmin = -1.,
            cmax = 1.,
            opacity = 1.
        )
    )

    # spike_x = go.Scatter3d(
    #     x = [filter_coefs[0, 0], filter_coefs[0, 0]], y = [filter_coefs[1, 0], filter_coefs[1, 0]], z = [filter_coefs[2, 0], z_offset],
    #     mode = "lines",
    #     line = dict(
    #         width = 2,
    #         color = "black"
    #     )
    # )

    # spike_y = go.Scatter3d(
    #     x = [filter_coefs[0, 0], filter_coefs[0, 0]], y = [filter_coefs[1, 0], filter_coefs[1, 0]], z = [filter_coefs[2, 0], z_offset],
    #     mode = "lines",
    #     line = dict(
    #         width = 2,
    #         color = "black"
    #     )
    # )

    # spike_z = go.Scatter3d(
    #     x = [filter_coefs[0, 0], x_offset], y = [filter_coefs[1, 0], filter_coefs[1, 0]], z = [filter_coefs[2, 0], filter_coefs[2, 0]],
    #     mode = "lines",
    #     line = dict(
    #         width = 2,
    #         color = "black"
    #     )
    # )

    scatter_trace = go.Scatter3d(
        x = np.repeat(filter_coefs[0, 0], repeats = 10), 
        y = np.repeat(filter_coefs[1, 0], repeats = 10), 
        z = np.repeat(filter_coefs[2, 0], repeats = 10), 
        mode = 'markers',
        marker = dict(
            size = 5,
            color = np.repeat(filter_coefs[3, 0], repeats = 10), 
            colorscale = 'Plasma',
            cauto = False,
            cmin = -1.,
            cmax = 1.,
            opacity = 0.5
        )
    )

    layout = go.Layout(
        title = "dataset of filter transmissions projected onto PCA basis space",
        height = 800,
        scene_xaxis_title = "basis_0",
        scene_xaxis_autorange = False,
        scene_xaxis_range = [-6., 0.5],
        scene_yaxis_title = "basis_1",
        scene_yaxis_autorange = False,
        scene_yaxis_range = [-1.5, 2.75],
        scene_zaxis_title = "basis_2",
        scene_zaxis_autorange = False,
        scene_zaxis_range = [-2.1, 1.3],
        showlegend = False
    )

fig = go.FigureWidget(data = [scatter_all, scatter_trace, scatter_pt], layout = layout)

def update(alpha1_slider, alpha2_slider, theta_slider):
    transmission_new = np.zeros(n_wav)

    if basis_2d:
        for w in range(n_wav):
            transmission_new[w] = transmission_mueller(gammas1[w], np.deg2rad(alpha1_slider), np.deg2rad(theta_slider))
    else:
        for w in range(n_wav):
            transmission_new[w] = transmission_mueller([gammas1[w], gammas2[w]], [np.deg2rad(alpha1_slider), np.deg2rad(alpha2_slider)], np.deg2rad(theta_slider))

    coefs_new, _, _, _ = np.linalg.lstsq(pca_basis.transpose(), transmission_new, rcond=None)

    with fig.batch_update():
        fig.data[-1].x = [coefs_new[0]]
        fig.data[-1].y = [coefs_new[1]]

        if basis_2d:
            fig.data[-1].marker.color = [coefs_new[0]]

            # fig.data[1].x = [coefs_new[0], coefs_new[0]]
            # fig.data[1].y = [coefs_new[1], y_offset]

            # fig.data[2].x = [coefs_new[0], x_offset]
            # fig.data[2].y = [coefs_new[1], coefs_new[1]]

            history_x = fig.data[4].x
            history_y = fig.data[4].y

            fig.data[4].x = np.append(history_x, coefs_new[0])[1:]
            fig.data[4].y = np.append(history_y, coefs_new[1])[1:]
            fig.data[4].marker.color = np.append(history_x, coefs_new[0])[1:]

        else:
            fig.data[-1].z = [coefs_new[2]]
            fig.data[-1].marker.color = [coefs_new[3]]

            # fig.data[1].x = [coefs_new[0], coefs_new[0]]
            # fig.data[1].y = [coefs_new[1], coefs_new[1]]
            # fig.data[1].z = [coefs_new[2], z_offset]

            # fig.data[2].x = [coefs_new[0], x_offset]
            # fig.data[2].y = [coefs_new[1], coefs_new[1]]
            # fig.data[2].z = [coefs_new[2], coefs_new[2]]

            # fig.data[3].x = [coefs_new[0], coefs_new[0]]
            # fig.data[3].y = [coefs_new[1], y_offset]
            # fig.data[3].z = [coefs_new[2], coefs_new[2]]

            history_x = fig.data[4].x
            history_y = fig.data[4].y
            history_z = fig.data[4].z
            history_c = fig.data[4].marker.color

            fig.data[4].x = np.append(history_x, coefs_new[0])[1:]
            fig.data[4].y = np.append(history_y, coefs_new[1])[1:]
            fig.data[4].z = np.append(history_z, coefs_new[2])[1:]
            fig.data[4].marker.color = np.append(history_c, coefs_new[3])[1:]

out = widgets.interactive_output(update, {'alpha1_slider': alpha1_slider, 'alpha2_slider': alpha2_slider, 'theta_slider': theta_slider})
display(ui)
fig

FigureWidget({
    'data': [{'marker': {'color': '#aaaaaa', 'opacity': 0.1, 'size': 2},
              'mode': …